# Python code of Gibbs sampling model for family test result Covid problem

In [1]:
# import neccessary libraries
import numpy as np
import random
import sympy as sym
from joblib import Parallel, delayed
from sympy import *
from sympy import symbols
from sympy.plotting import plot, plot3d
from datetime import datetime
from scipy.stats import beta
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

In [2]:
# function to count the number of the infected neighbores of i at t:
def CNbr(G,X,n,T):
    C=np.zeros((T,n))
    for t in range(T):
        C[t]=G[t].dot(X.T[t])
    return C.T

In [3]:
# Function to obtain the very initial sample of X, using forwad sampling:
def Forward_Sampling(T,n,G,F,param):
    
    alpha_=param[0]
    bata_=param[1]
    bataf=param[2]
    gama_=param[3]
    p0=P
    p1=1-P
    x=int(np.round(((1-P)*n),0))
    X=np.zeros((n,T))  
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    infR=CNbr(G,X,n,T)
    for t in range(T-1):
        cf=F.dot(X.T[t])
        for i in range(n):
            c=infR[i,t]
            if X[i,t]==0:
                p0=(1-alpha_-beta_*c-betaf*cf[i])
                p1=(alpha_+beta_*c+betaf*cf[i])
            else:
                p0=(gama_)
                p1=(1-gama_)
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 
    return X

In [44]:
def Sample_hidden_state(X,G,F,unique_rows,Y,param,t):
    
    alpha_=param[0]
    beta_=param[1]
    bataf=param[2]
    gama_=param[3]
    theta_0_=param[4]
    thata_1_=param[5]
    
       
    for i in range(n):
        p_0,p_1=0,0
        if t==0:
            p_0,p_1=P,1-P
        else:
            p_0,p_1=1,1
        pow0=1-Y[i,t]
        pow1=Y[i,t]
        number_of_members_in_family=np.sum(unique_rows[family_index(i,unique_rows)])
        
        X[i,t]=0
        number_of_infected_members_in_family0=F.dot(X.T[t])[i]
        number_of_healthy_members_in_family0=number_of_members_in_family-number_of_infected_members_in_family0
        p_0=(1/number_of_members_in_family)*p_0*((1-theta_0_)*number_of_healthy_members_in_family0+(1-theta_1_)*number_of_infected_members_in_family0)**pow0*(theta_1_*number_of_infected_members_in_family0+theta_0_*number_of_healthy_members_in_family0)**pow1
        
        X[i,t]=1
        number_of_infected_members_in_family1=F.dot(X.T[t])[i]
        number_of_healthy_members_in_family1=number_of_members_in_family-number_of_infected_members_in_family1
        p_1=(1/number_of_members_in_family)*p_1*((1-theta_0_)*number_of_healthy_members_in_family1+(1-theta_1_)*number_of_infected_members_in_family1)**pow0*(theta_1_*number_of_infected_members_in_family1+theta_0_*number_of_healthy_members_in_family1)**pow1
        
        X[i,t]=0
        if (t==0):
            c=G[t].dot(X.T[t])[i]
        else:    
            c=G[t-1].dot(X.T[t-1])[i]
        if t!=T-1:
            if X[i,t+1]==0:
                p_0=p_0*(1-alpha_-beta_*c-betaf*number_of_infected_members_in_family0)
                p_1=p_1*gama_
            else:
                p_0=p_0*(alpha_+beta_*c+betaf*number_of_infected_members_in_family0)
                p_1=p_1*(1-gama_)

        X[i,t]=1
        if (t==0):
            c=G[t].dot(X.T[t])[i]
        else:    
            c=G[t-1].dot(X.T[t-1])[i]
        if t!=T-1:
            if X[i,t+1]==0:
                p_0=p_0*(1-alpha_-beta_*c-betaf*number_of_infected_members_in_family1)
                p_1=p_1*gama_
            else:
                p_0=p_0*(alpha_+beta_*c+betaf*number_of_infected_members_in_family1)
                p_1=p_1*(1-gama_)
        
        family_members=unique_rows[family_index(i,unique_rows)]
        if t!=T-1:        
            X[i,t]=0
            for j in np.where(family_members==1)[0]:
                p_0=p_0*transition(X,t,G,F,j,params)
            for j in np.where(G[t][i]==1)[0]:
                p_0=p_0*transition(X,t,G,F,j,params)
            X[i,t]=1
            for j in np.where(family_members==1)[0]:
                p_1=p_1*transition(X,t,G,F,j,params)
            for j in np.where(G[t][i]==1)[0]:
                p_1=p_1*transition(X,t,G,F,j,params)
               
                
        if p_0+p_1==0:            
            l=0.5
        else:
            l=p_1/(p_0+p_1)
        X[i,t]=np.random.binomial( 1,  l,size=None)    
        
    return X    

In [30]:
def transition(X,t,G,F,j,param):
    alpha_=param[0]
    bata_=param[1]
    bataf=param[2]
    gama_=param[3]
    c=G[t].dot(X.T[t])[j]
    number_of_infected_members_in_family=F.dot(X.T[t])[j]
    k=X[j,t]-2*X[j,t+1]
    if k==0:
        return 1-alpha_-beta_*c-betaf*number_of_infected_members_in_family
    elif k==-2:
        return alpha_+beta_*c+betaf*number_of_infected_members_in_family
    elif k==1:
        return gama_
    else:
        return 1-gama_
    

In [ ]:
A=X[np.where(family_members==1),t]-2*X[np.where(family_members==1),t+1]
            pow00=np.count_nonzero(A==0)
            pow11=np.count_nonzero(A==-1)
            pow10=np.count_nonzero(A==1)
            pow01=np.count_nonzero(A==-2)
            p_0=p_0*stay_healthy0**pow00*become_infected0**pow01*stay_infected**pow11*recovered**pow10
            
            X[i,t]=1
            B=X[np.where(family_members==1),t]-2*X[np.where(family_members==1),t+1]
            pow00=np.count_nonzero(B==0)
            pow11=np.count_nonzero(B==-1)
            pow10=np.count_nonzero(B==1)
            pow01=np.count_nonzero(B==-2)
            p_1=p_1*stay_healthy1**pow00*become_infected1**pow01*stay_infected**pow11*recovered**pow10            
            X[i,t]=0
            state_transition=X.T[t]+2*X.T[t+1]+1
            key=np.multiply(G[t][i],state_transition)
            pow_stay_healthy=pow_stay_healthy+np.count_nonzero(key == 4)
            pow_become_infected=pow_become_infected+np.count_nonzero(key == 3)
            pow_recovery=pow_recovery+np.count_nonzero(key == 2)
            pow_stay_infected=pow_stay_infected+np.count_nonzero(key ==1)
            p0=p_0*(stay_healthy_0)**pow_stay_healthy*(become_infected_0)**pow_become_infected*recovered**pow_recovery*stay_infected**pow_stay_infected
            X[i,t]=1
            state_transition=X.T[t]+2*X.T[t+1]+1
            key=np.multiply(G[t][i],state_transition)
            pow_stay_healthy=pow_stay_healthy+np.count_nonzero(key == 4)
            pow_become_infected=pow_become_infected+np.count_nonzero(key == 3)
            pow_recovery=pow_recovery+np.count_nonzero(key == 2)
            pow_stay_infected=pow_stay_infected+np.count_nonzero(key ==1)
            p1=p_1*(stay_healthy_1)**pow_stay_healthy*(become_infected_1)**pow_become_infected*recovered**pow_recovery*stay_infected**pow_stay_infected


In [6]:
# Gibbs sampling to obtain X, as new sample of posterior distribution:
def Calculate_X(K,T,X,G,F1,Y,param):
    for k in range(K):
        for t in range(T):
            X=Sample_hidden_state(X,G,F1,unique_rows,Y,param,t)
    return X                

# Functions to sample from beta distribution


In [7]:
# function to sample new parameters and update parameters:
def Params(R,G,F,X,n,T,Y,param):
    
    alpha_=param[0]
    bata_=param[1]
    bataf=param[2]
    gama_=param[3]
    theta_0_=param[4]
    thata_1_=param[5]    
   
    TP=np.sum(np.multiply(unique_rows.dot(X),YF))
    FP=np.count_nonzero(unique_rows.dot(X)-YF==-1)
    
    infR=np.array(CNbr(G,X,n,T))
    alpha_=Sample_alpha(a_alpha +  np.count_nonzero(R==0) , b_alpha +np.count_nonzero(X==0)- np.count_nonzero(R==0))
    beta_=Sample_beta(a_beta + np.count_nonzero(R==2) , b_beta +np.sum(np.multiply((1-X),infR))-np.count_nonzero(R==2))
    betaf=Sample_betaf(a_betaf + np.count_nonzero(R==3) , b_betaf +np.sum(np.multiply((1-X),F.dot(X)))-np.count_nonzero(R==3))
    gama_=Sample_gama(a_gama +np.count_nonzero((X[:,:-1]-X[:,1:])==1), b_gama+np.sum(X)-np.count_nonzero((X[:,:-1]-X[:,1:])==1))
    theta_0_=Sample_theta0( a_teta0+FP,b_teta0+np.count_nonzero((unique_rows.dot(X))==0)-FP)
    theta_1_=Sample_theta1( a_teta1+TP,b_teta1+np.sum(unique_rows.dot(X))-TP)
    
    R=np.zeros((n,T))+1
    for i in range(n):
        for t in range(T-1):
            infr=int(infR[i,t])
            cf=int(F.dot(X.T[t])[i])
            pr_a=alpha_/(alpha_+beta_*infr+betaf*cf)
            pr_b=beta_/(alpha_+beta_*infr+betaf*cf)
            pr_bf=betaf/(alpha_+beta_*infr+betaf*cf)
            v=np.random.multinomial(1, [pr_a]+[pr_b]*infr+[pr_bf]*cf)
            if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                elif (cf!=0):
                    if ((np.equal(1,v[-cf:])).any):
                        R[i,t]=3
                else:    
                    R[i,t]=2
    param=[alpha_,beta_,betaf,gama_,theta_0_,theta_1_]
    return param,R

In [8]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=10000):
        if (i>0.001)&(i<0.091):
            alpha_=round(i,3)
            break
    return alpha_        


In [9]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=10000):
        if (i>0.001)&(i<0.0451):
            beta_=round(i,4)
            break
    return beta_        

In [10]:
def Sample_betaf(a_betaf, b_betaf):
    for i in beta.rvs(a_betaf, b_betaf, size=10000):
        if (i>0.001)&(i<0.091):
            betaf=round(i,4)
            break
    return betaf        

In [11]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=10000):
        if (i>0.1)&(i<0.7):
            gama_=round(i,3)
            break
    return gama_  

In [12]:
def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        print(i,a_teta0, b_teta0)
        if (i>0.001)&(i<0.51):
            theta_0_=round(i,3)
            break
    return theta_0_  

In [13]:
def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=10000):
        print(i,a_teta1, b_teta1)
        if i>0.6:
            theta_1_=round(i,3)
            break
    return theta_1_  

In [14]:
# initialize parameters for beta distributions:
a_alpha=1
b_alpha=10
a_beta=1
b_beta=5
a_betaf=1
b_betaf=6
a_gama=1
b_gama=2
a_teta0=1
b_teta0=1000
a_teta1=1000
b_teta1=1
P=.8

In [15]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
betaf=Sample_betaf(a_betaf, b_betaf)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)

0.0012593259538197848 1 1000
0.999001581119122 1000 1


In [16]:
params=np.array([alpha_,beta_,betaf,gama_,theta_0_,theta_1_])
params

array([0.009 , 0.0393, 0.0804, 0.541 , 0.001 , 0.999 ])

In [17]:
# funtion to retun related family index of individual i:
def family_index(i,unique_rows):
    for j in range(n):
        if unique_rows[j,i]==1:
            return j

In [18]:
# Function to generates Synthetic dataset
def Synthetic_Data(n,T,y,params,number_families):
    alpha_,beta_,betaf,gama_,theta_0_,theta_1_=params[0],params[1],params[2],params[3],params[4],params[5]
    x=int(np.round(((1-P)*n),0))
    X=np.zeros((n,T))
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    # Random social network
    G=[]
    for j in range(T):
        g=np.identity(n,dtype=int)
        for i in range(n):
            inx=np.random.choice(range(i,n), y)
            g[i,inx]=1  
            g[inx,i]=1
        G.append(g)
    G=np.array(G)
    
    # Synthetize Family matrix:
    a=list(range(n))
    Family=np.identity(n,dtype=int)
    for i in range(1,number_families):
        number_family_members=np.random.randint(2, 5)
        inx=random.sample(list(a), number_family_members)
        a=set(a).difference(inx)
        for j in inx:
            for k in inx:
                Family[j,k]=1  
    G=G-Family
    Family=Family-np.identity(n,dtype=int)
    G = np.where(G <0, 0, G)
    # Synthetize X, using params,G, Family and transition probability:
    
    for t in range(T-1):
        for i in range(n):
            infR=CNbr(G,X,n,T)
            cf=int(Family.dot(X.T[t])[i])
            c=infR[i,t]
            if X[i,t]==0:
                p0=1-alpha_-beta_*c-betaf*cf
                p1=alpha_+beta_*c+betaf*cf
            else:
                p0=gama_
                p1=1-gama_
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            if (l<0)|(l>1):
                print(p0,p1,l)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 

    # Synthetize Y, using params,G, X, emission probability:
    Family=Family+np.identity(n,dtype=int)
    unique_rows = np.unique(Family, axis=0)
    nf=unique_rows.shape[0]
    YF=np.zeros((nf,T))
    for t in range(T):
        for i in range(nf):
            number_of_members_in_family=np.sum(unique_rows[i])
            number_of_infected_members_in_family=unique_rows[i].dot(X.T[t])
            number_of_healthy_members_in_family= number_of_members_in_family-number_of_infected_members_in_family
            py1=(theta_0_*number_of_healthy_members_in_family+theta_1_*number_of_infected_members_in_family)/ number_of_members_in_family
            py0=((1-theta_0_)*number_of_healthy_members_in_family+(1-theta_1_)*number_of_infected_members_in_family)/ number_of_members_in_family
            l=py1/(py1+py0)
            YF[i,t]=np.random.binomial( 1, l,size=None) 
    return G,YF,X,Family           

In [51]:
# Generate synthetic data,G ,Y:
n,T,y=100,100,10
number_families=10
synthetic_data=Synthetic_Data(n,T,y,params,number_families)
G,YF,X,Family =synthetic_data[0],synthetic_data[1],synthetic_data[2],synthetic_data[3]

In [52]:
# Save true value of X in Z, as the correct label of data:
Z=X
np.sum(Z)

2838.0

In [53]:
F=Family

In [54]:
unique_rows = np.unique(F.T, axis=0)

In [55]:
Y=np.zeros((n,T))
for i in range(n):
    for j in range(unique_rows.shape[0]):
        if unique_rows[j,i]==1:
            Y[i,:]=YF[j,:]

In [56]:
# define auxiliary variable R(n,t):
infR=np.array(CNbr(G,X,n,T))
R=np.zeros((n,T))+1
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        cf=int(F.dot(X.T[t])[i])
        pr_a=alpha_/(alpha_+beta_*infr+betaf*cf)
        pr_b=beta_/(alpha_+beta_*infr+betaf*cf)
        pr_bf=betaf/(alpha_+beta_*infr+betaf*cf)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr+[pr_bf]*cf)
        if (X[i][t]==0)&(X[i][t+1]==1):
            if v[0]==1:
                R[i,t]=0
            elif ((v[-cf:]==1).any())&(cf!=0): 
                R[i,t]=3
            else:    
                R[i,t]=2


In [57]:
# Run Gibbs sampling algorithm to estimate X:
U=1
K=1
X=Forward_Sampling(T,n,G,F,params)
X=Calculate_X(K,T,X,G+np.identity(n,dtype=int),F,Y,params)
winsound.Beep(freq, duration)

In [58]:
# define auxiliary variable R(n,t):
infR=np.array(CNbr(G,X,n,T))
R=np.zeros((n,T))+1
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        cf=int(F.dot(X.T[t])[i])
        pr_a=alpha_/(alpha_+beta_*infr+betaf*cf)
        pr_b=beta_/(alpha_+beta_*infr+betaf*cf)
        pr_bf=betaf/(alpha_+beta_*infr+betaf*cf)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr+[pr_bf]*cf)
        if (X[i][t]==0)&(X[i][t+1]==1):
            if v[0]==1:
                R[i,t]=0
            elif (cf!=0):
                if ((v[-cf:]==1).any()):
                    R[i,t]=3
            else:    
                R[i,t]=2


In [59]:
X

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 1.],
       ...,
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 1., 1., 1.]])

In [60]:
np.sum(Z)

2838.0

In [61]:
# Main code to run entire Gibbs algorithm U times:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    print("Current Parameters:",params)
    prm=Params(R,G,F,X,n,T,Y,params)
    params=prm[0]
    X=Calculate_X(K,T,X,G,F,Y,params)
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration) 
print("Current Parameters:",params)

******************* Iteration: 0 *****************************************************************************
Current Time is : 18:23:02
Current Parameters: [0.091, 0.0264, 0.004, 0.533, 0.001, 0.987]
0.0014988431442750781 10 6469
0.8907003928734919 3501.0 449.0
Verctor of Health States: 
 [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 1. 1.]
 ...
 [0. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 0.]]
Current Parameters: [0.013, 0.0352, 0.0854, 0.535, 0.001, 0.891]


In [62]:
np.sum(X)

3229.0

In [ ]:
# synthetized observation vectore:
Y_=np.zeros((n,T))
for t in range(T):
    for i in range(n):
        if X[i,t]==0:
            Y_[i,t]=np.random.binomial( 1, theta_0_,size=None) 
        else:
            Y_[i,t]=np.random.binomial( 1, theta_1_,size=None) 

In [63]:
print(np.sum(X),np.sum(Z),np.count_nonzero(X-Z))

3229.0 2838.0 1259


In [ ]:
Y=Y_

# The following is for family test result problem:

In [ ]:
# Synthetize data for famity test result problem

In [ ]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=1000):
        if (i>0.029)&(i<0.051):
            alpha_=round(i,3)
            break
    return alpha_        


In [ ]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=10000):
        if (i>0.00491)&(i<0.01451):
            beta_=round(i,4)
            break
    return beta_        


In [ ]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=1000):
        if (i>0.291)&(i<0.31):
            gama_=round(i,3)
            break
    return gama_  


In [ ]:
def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        print(i)
        if (i<0.19975):
            theta_0_=round(i,3)
            break
    return theta_0_  


In [ ]:
def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=100000):
        if .9901>i>0.8:
            theta_1_=round(i,3)
            break
    return theta_1_  


In [ ]:
# initialize parameters for beta distributions:
a_alpha=2
b_alpha=7
a_beta=2
b_beta=10
a_gama=1
b_gama=3
a_teta0=.1
b_teta0=500
a_teta1=4000
b_teta1=100

In [ ]:
# Choose some params close to obtimized params:
alpha_= 0.05
beta_= 0.01
gama_= 0.3
theta_0_= 0.07
theta_1_= 0.8
params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])

In [ ]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)

In [ ]:
params=np.array([alpha_,beta_,gama_,theta_0_,theta_1_])
params

In [ ]:
#Use uptimized params:
number_families=4
alpha_=params[0]
bata_=params[1]
gama_=params[2]
theta_0_=params[3]
thata_1_=params[4]    
p_=0.9

In [ ]:
# Synthetize the "Family" matrix, which denotes that each two individuals are in the same family or not:
a=list(range(n))
Family=np.identity(n,dtype=int)
for i in range(1,number_families):
    number_family_members=np.random.randint(2, 5)
    inx=random.sample(list(a), number_family_members)
    a=set(a).difference(inx)
    for j in inx:
        for k in inx:
            Family[j,k]=1
F=Family
F1=F-np.identity(n, dtype=int)

In [ ]:
# Synthetize G,Y,X:
n,T,y=10,10,2
synthetic_data=Synthetic_Data(n,T,y,params)
G,Y,X =synthetic_data[0],synthetic_data[1],synthetic_data[2]

# Simulation data is ready now, YF,G,Family:

In [ ]:
F

In [ ]:
G[1]

In [ ]:
# ایده اول برای مین فیلد و شبه مشاهدات

In [ ]:
y=np.zeros((n,T))
for i in range(n):
    for j in range(n):
         if unique_rows[j,i]==1:
                break
    for t in range(T):
       
        y[i,t]=int(YF[j,t]-np.dot(np.multiply(unique_rows[j],X.T[t]),W)+X[i,t]*W[i])
        y = np.where(y <0, 0, y)
y 

In [ ]:
params

In [ ]:
# Run Forward sampling algorithm to estimate X:
U=10
J=100
P=p_
X=Forward_Sampling(T,n,G,params)
X=Calculate_X(J,T,n,X,G,y,params)
winsound.Beep(freq, duration)

In [ ]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [ ]:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    prm=Params(R,G,X,n,T,y,params)
    params=prm[0]
    print("updated parameters:",params)
    X=Calculate_X(J,T,n,X,G,y,params)
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration)

In [ ]:
for i in range(n):
    for t in range(T):
        if X[i,t]==0:
            yy[i,t]=np.random.binomial( 1, theta_0_,size=None) 

In [ ]:
print(np.sum(X),np.sum(Z),np.count_nonzero(Z-X))

In [ ]:
# ایده دوم برای مین فیلد و شبه مشاهدات

In [ ]:
# Save true X in Z:
Z=X

In [ ]:
#initialize parameters a0,b0,a1,b1:
A=np.round(np.random.uniform(0, 10, (4,1)),3)
a0,b0,a1,b1=A[0][0],A[1][0],A[2][0],A[3][0]
print(a0,b0,a1,b1)

In [ ]:
def Sample_Sudo_theta0(a0, b0):
    for i in beta.rvs(a0, b0, size=10000):
        if (i>0.001)&(i<0.5):
            theta0=round(i,3)
            break
    return theta0  

In [ ]:
def Sample_Sudo_theta1(a1, b1):
    for i in beta.rvs(a1, b1, size=100000):
        if .9901>i>0.5:
            theta1=round(i,3)
            break
    return theta1  


In [ ]:
theta0=Sample_Sudo_theta0(a0, b0)
theta0

In [ ]:
theta1=Sample_Sudo_theta1(a1, b1)
theta1

In [ ]:
y=np.zeros((n,T))


In [ ]:
def Calculate_Sudo_observations(a0,b0,a1,b1,X):
    for i in range(n):
        j=family_index(i,unique_rows)
        for t in range(T):
            if YF[j,t]==1:
                theta1=Sample_Sudo_theta1(a1+YF[j,t]*np.sum(unique_rows[j])-np.sum(X[unique_rows[j]==1,t])+X[i,t], b1+np.sum(X[unique_rows[j]==1,t])-X[i,t])
                y[i,t]=np.random.binomial( 1, theta1,size=None)
            else:
                y[i,t]=0
    return y

In [ ]:
# Run Forward sampling algorithm to estimate X:
U=10
J=1000
P=p_
X=Forward_Sampling(T,n,G,params)
y=Calculate_Sudo_observations(a0,b0,a1,b1,X)
X=Calculate_X(J,T,n,X,G,y,params)
winsound.Beep(freq, duration)

In [ ]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [ ]:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    prm=Params(R,G,X,n,T,y,params)
    params=prm[0]
    print("updated parameters:",params)
    y=Calculate_Sudo_observations(a0,b0,a1,b1,X)
    X=Calculate_X(J,T,n,X,G,y,params)
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration)
    
    

In [ ]:
params

In [ ]:
#y=Calculate_Sudo_observations(a0,b0,a1,b1,X)
for i in range(n):
    for t in range(T):
        j=family_index(i,unique_rows)
        if YF[j,t]==1:
            if (np.sum(X[unique_rows[j]==1,t]))==0:
                print(np.sum(X[unique_rows[j]==1,t]))

In [ ]:
Y=np.zeros((n,T))
for i in range(n):
    for j in range(unique_rows.shape[0]):
        if unique_rows[j,i]==1:
            Y[i,:]=YF[j,:]

In [ ]:
np.sum(y)

In [ ]:
print(np.sum(X),np.sum(Z),np.count_nonzero(Z-X))

In [ ]:
# sample parameter of sudo obsarvations:
def Sample_theta(a, b):
    for i in beta.rvs(a, b, size=10000):
        if (i>0.3)&(i<0.6):
            theta=round(i,3)
            break
    return theta 